In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
'''
# ---------------------------------------------------------------------------------------------------------------------------
General Information:
--------------------
Task 4 -- Introduction to machine learning -- Taste Close?
Authors: Dejan and Sindi

General approach description:
-----------------------------
Use a pretrainded model of PyTorch/Tenserflow Keas. I.e. following the link: https://pytorch.org/vision/stable/models.html 
and https://www.tensorflow.org/tutorials/images/transfer_learning. This will allow to use an already good CNN model. 
With the pretrained model, we want to extract the features created by the model. The extracted features can then be used 
to train an classifier, depending on how close the data: I.e. how close the first image A is to B and C. 

Most of the code is based on the introduction to siamese networks: https://keras.io/examples/vision/siamese_network/
Howeverr, need to make it more efficient as it takes too long... 

Important links
---------------
1)  Triamese Networks/ Transfer Learning:
    https://medium.com/@crimy/one-shot-learning-siamese-networks-and-triplet-loss-with-keras-2885ed022352

1)  Preparing the data:
    https://keras.io/examples/vision/siamese_network/

2)  How to work with with shuffle, repeat and batch of data sets. 
    https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset

3)  Building image pipelines with tensorflow
    https://cs230.stanford.edu/blog/datapipeline/

4)  working with data papelines
    https://stackoverflow.com/questions/56399919/proper-way-to-iterate-tf-data-dataset-in-session-for-2-0

5)  Dissimmilarity similarity measureas 
    https://towardsdatascience.com/17-types-of-similarity-and-dissimilarity-measures-used-in-data-science-3eb914d2681
# ---------------------------------------------------------------------------------------------------------------------------
'''


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import resnet
import itertools
import sys

# connected to GPU????
# ---------------------------------------------------------------------------------------------------------------------------
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


img_size = (224,224)
img_shape = img_size + (3,)


# HELPER FUNCTIONS and STUFF
# ---------------------------------------------------------------------------------------------------------------------------
def preprocess_image(filename):
    '''
    preprocessing_image() was inspired by: https://keras.io/examples/vision/siamese_network/
    Loades the specified file as a JPEG image, prerocess it by applying rescaling into [0, 1] from [0,255]. Converts it 
    into float32 and resizes it into image_size. (160, 160). Further data augmentation is done by flipping it ranodomly 
    either to left or right. Convert it into [0, 1] as the base model is set up with this values. 
    '''
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels = 3)

    rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
    image = tf.cast(image, tf.float32)
    image = rescale(image)

    image = tf.image.resize(image, img_size)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image



def preprocess_test_image(filename):
    '''
    preprocessing_image was inspired by: https://keras.io/examples/vision/siamese_network/
    Loades the specified file as a JPEG image, prerocess it by applying rescaling into [0, 1] from [0,255]. Converts it 
    into float32 and resizes it into image_size. (160, 160). Further data augmentation is done by flipping it ranodomly 
    either to left or right. 
    '''
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels = 3)

    rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
    image = tf.cast(image, tf.float32)
    image = rescale(image)

    image = tf.image.resize(image, img_size)
    return image


def preprocess_test_triplets(anchor_id, positive_id, negative_id):
    '''
    preprocessing_image was inspired by: https://keras.io/examples/vision/siamese_network/
    Given the filenames corresponding to the three images, load and preprocess them.
    '''
    anchor = preprocess_test_image(create_path(anchor_id))
    positive = preprocess_test_image(create_path(positive_id))
    negative = preprocess_test_image(create_path(negative_id))

    return tf.stack([anchor, positive, negative], axis = 0)


def create_path(triplet_id):
    '''
    Needed to make it easier to load pictures
    creates the path to the food folder for each line....
    '''
    return "gdrive/MyDrive/food/" + triplet_id + ".jpg"


def preprocess_triplets(anchor_id, positive_id, negative_id):
    '''
    loading the individual triplets.
    Given the filenames corresponding to the three images, load and preprocess them.
    '''
    anchor = preprocess_image(create_path(anchor_id))
    positive = preprocess_image(create_path(positive_id))
    negative = preprocess_image(create_path(negative_id))

    return tf.stack([anchor, positive, negative], axis = 0), 1


def get_loss(pseudo_target, image_features):
    anchor = image_features[..., 0]
    positive = image_features[..., 1]
    negative = image_features[..., 2] 

    ap_distance = tf.reduce_sum(tf.square(anchor - positive), 1)
    an_distance = tf.reduce_sum(tf.square(anchor - negative), 1)
    loss = tf.reduce_mean(tf.math.softplus(ap_distance - an_distance))

    return loss


def get_accuracy(pseudo_target, image_features):

    anchor = image_features[..., 0]
    positive = image_features[..., 1]
    negative = image_features[..., 2] 

    ap_distance = tf.reduce_sum(tf.square(anchor - positive), 1)
    an_distance = tf.reduce_sum(tf.square(anchor - negative), 1)
    
    acc = tf.reduce_mean(tf.cast(tf.greater_equal(an_distance, ap_distance), tf.float32))
    return acc

# MAIN CODE PART COMBINING EVERYTHING TOGETHER...
# ---------------------------------------------------------------------------------------------------------------------------
def main():
    '''
    importing and splitting the data into trainig and validation set. this can be expanedd to Cross Validation if 
    the trained model does not take too long to fit the data.
    ''' 
    # -----------------------------------------------------------------------------------------------------------------------
    n_train = 59515
    n_batch = 64
    n_epoch = 5
    fine_tune_epochs = 1
    n_buffer = 1024
    base_learning_rate = 1e-3 # in order to have the relation steps_per_epoch * batch_size = number_of_rows_in_train_data
    n_step_per_epoch =  np.ceil(n_train / n_batch)
    finetuning = True
    fine_tune_at = 135


    # Creating the datapipeline for the model: Keras Models will iterate without creating a loop through all of the batches
    # -----------------------------------------------------------------------------------------------------------------------
    AUTOTUNE = tf.data.experimental.AUTOTUNE 
    trips_train = tf.data.TextLineDataset('gdrive/MyDrive/train_triplets.txt')
    trips_train = trips_train.map(
        lambda trip: preprocess_triplets(tf.strings.split(trip)[0],
                                         tf.strings.split(trip)[1],
                                         tf.strings.split(trip)[2]), num_parallel_calls = AUTOTUNE)

    trips_test = tf.data.TextLineDataset('gdrive/MyDrive/test_triplets.txt')
    trips_test = trips_test.map(
        lambda trip: preprocess_test_triplets(tf.strings.split(trip)[0], 
                                              tf.strings.split(trip)[1], 
                                              tf.strings.split(trip)[2]), num_parallel_calls = AUTOTUNE)

    trips_train = trips_train.shuffle(buffer_size = n_buffer, seed = 777,
            reshuffle_each_iteration = True).repeat().batch(n_batch, drop_remainder = False)



    # get the basemodel. also prepare it for fine tuning. 
    # -----------------------------------------------------------------------------------------------------------------------
    base_cnn = tf.keras.applications.MobileNetV2(weights = 'imagenet', 
                                                 input_shape = img_shape, 
                                                 include_top = False)
    base_cnn.trainable = False
    in_shape = (3, ) + img_shape    
    features_machine = tf.keras.Sequential([tf.keras.layers.GlobalAveragePooling2D(), 
        tf.keras.layers.Dense(128), 
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Lambda(lambda b_norm: tf.math.l2_normalize(b_norm, axis=1))])
    
    # from inputs till output for model
    inputs = tf.keras.Input(shape=in_shape)
    base_cnn_anchor = base_cnn(inputs[:, 0, ...])
    base_cnn_positive = base_cnn(inputs[:, 1, ...])
    base_cnn_negative = base_cnn(inputs[:, 2, ...])
    feature_mach_anchor = features_machine(base_cnn_anchor)
    feature_mach_positive = features_machine(base_cnn_positive)
    feautre_mach_negative = features_machine(base_cnn_negative)

    outputs = tf.stack([feature_mach_anchor, feature_mach_positive, feautre_mach_negative], axis = -1)

    features_model = Model(inputs = inputs, outputs = outputs)
    features_model.summary()


    # Modelfitting
    # -----------------------------------------------------------------------------------------------------------------------
    print("# -------------------------------------------- MODEL FITTING -------------------------------------------- #")
    features_model.compile(optimizer=optimizers.Adam(base_learning_rate), loss = get_loss, metrics = [get_accuracy])
    history = features_model.fit(trips_train, 
                                 epochs = n_epoch, 
                                 steps_per_epoch = n_step_per_epoch)


    # FineTuning
    # -----------------------------------------------------------------------------------------------------------------------
    print("# -------------------------------------------- FINE TUNING -------------------------------------------- #")
    if finetuning:
        base_cnn.trainable = True
        for layer in base_cnn.layers[:fine_tune_at]:
            layer.trainable =  False

        features_model.compile(loss=get_loss,
                  optimizer = tf.keras.optimizers.RMSprop(base_learning_rate/10),
                  metrics=[get_accuracy])
        features_model.summary()
        features_model.fit(trips_train, 
                           epochs = int(n_epoch + fine_tune_epochs),
                           initial_epoch=history.epoch[-1],
                           steps_per_epoch = n_step_per_epoch)


    # Predicitions
    # -----------------------------------------------------------------------------------------------------------------------
    print("# -------------------------------------------- PREDICTING ITERATIONS -------------------------------------------- #")
    trips_test = trips_test.batch(256).prefetch(2)
    predictions = []
    i = 0

    for x in trips_test:
        sample = x
        embeddings = features_model(sample)
        anchor_test = embeddings[..., 0]
        positive_test = embeddings[..., 1]
        negative_test = embeddings[..., 2] 

        ap_distance_test = tf.reduce_sum(tf.square(anchor_test - positive_test), 1)
        an_distance_test = tf.reduce_sum(tf.square(anchor_test - negative_test), 1)

        predictions.append(tf.cast(tf.greater_equal(an_distance_test, ap_distance_test), tf.int8).numpy().tolist())
        print("Predicition Iteration: {}".format(i))
        i = i + 1


    predictions = pd.DataFrame(np.array(list(itertools.chain(*predictions))).reshape(-1, 1))
    predictions.to_csv('gdrive/MyDrive/binary.csv', header = None, index = False)

    # FINISH
    # -----------------------------------------------------------------------------------------------------------------------
    print("# -------------------------------------------- FINISHED AND SAVED -------------------------------------------- #")
                  


if __name__ == '__main__':
    main()








Found GPU at: /device:GPU:0
9412608/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 3, 224, 224, 0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None, 224, 224, 3)  0           input_2[0][0]                    
___